In [1]:
import numpy as np
import pandas as pd

In [2]:
# def calc_info_gain(data,classes,feature):
#     gain = 0
#     nData = len(data)
#     # List the values that feature can take
#     values =[]
#     for datapoint in data:
#         if datapoint[feature] not in values:
#             values.append(datapoint[feature])
    
#     featureCounts = np.zeros(len(values))
#     entropy = np.zeros(len(values))
#     valueIndex = 0
#     #Find where those values appear in data[feature] and the corresponding class
#     for value in values:
#         dataIndex = 0
#         newClasses = []
#         for datapoint in data:
#             if datapoint[feature] == value:
#                 featureCounts[valueIndex]+=1
#                 newClasses.append(classes[dataIndex])
#             dataIndex +=1
#         # Get the values in newClasses
#         classValues = []
#         for aclass in newClasses:
#             if classValues.count(aclass)==0:
#                 classValues.append(aclass)
#         classCounts = np.zeros(len(classValues))
#         classIndex = 0
#         for classValue in classValues:
#             for aclass in newClasses:
#                 if aclass == classValue:
#                     classCounts[classIndex] +=1
#             classIndex +=1
            
#         for classIndex in range(len(classValues)):
#             entropy[valueIndex] += calc_entropy(float(classCounts[classIndex])/sum(classCounts))
#         gain += float(featureCounts[valueIndex])/nData*entropy[valueIndex]
#         valueIndex += 1
#     return gain


In [3]:
# def findPath(graph,start,end,pathSoFar):
#     pathSoFar = pathSoFar + [start]
#     if start == end:
#         return pathSoFar
#     if start not in graph:
#         return None
#     for node in graph[start]:
#         if node not in pathSoFar:
#             newpath = findPath(graph,node,end,pathSoFar)
#             return newpath
#     return None

In [4]:
# def make_tree(data,classes,featureNames):
#     #Various initialisations suppressed
#     default = classes[np.argmax(frequency)]
#     if nData ==0 or nFeatures ==0:
#         #Have reach empty branch
#         return default
#     elif classes.count(classes[0]) == nData:
#         #Only one class remain
#         return classes[0]
#     else:
#         #Choose which feature is best
#         gain = np.zeros(nFeatures)
#         for feature in range(nFeatures):
#             g = calc_info_gain(data,classes,feature)
#             gain[feature] = totalEntropy -g
#         bestFeature = np.argmax(gain)
#         tree = {featureNames[bestFeature]:{}}
#         #Find the possible feature values
#         for value in values:
#             # Find the datapoints with each feature value
#             for datapoint in data:
#                 if datapoint[bestFeature] == value:
#                     if bestFeature ==0:
#                         datapoint = datapoint[1:]
#                         newnames = featureNames[1:]
#                     elif bestFeature == nFeatures:
#                         datapoint = datapoint[:-1]
#                         newNames = featureNames[:-1]
#                     else:
#                         datapoint = datapoint[:bestFeature]
#                         datapoint.extend(datapoint[bestFeature+1:])
#                         newNames = featureNames[:bestFeature]
#                         newNames.extend(featureNames[bestFeature+1:])
#                     newData.append(datapoint)
#                     newClasses.append(classes[index])
#                 index+=1
#             #Now recurse to the next level
#             subtree = make_tree(newData,newClasses,newNames)
#             # And on returning, add the subtree on to the tree
#             tree[featureNames[bestFeature]][value] = subtree
#         return tree

In [5]:
class Node:
    def __init__(self,index,classes,label,depth=0,attribute=None,valueNames=None):
        self.feature = attribute
        self.values = valueNames
        self.index = index
        self.classes = classes
        self.valueNode = {}
        self.depth = depth
        
    def subnode(self, node,value):
        self.valueNode[value] = node


In [6]:
class Tree:
    
    def __init__(self,data,classes,label,max_depth):
        self.head = Node(data.index,classes,label)
        self.leaves = [self.head]
        self.max_depth = max_depth
        
    
    def branch(self,leaf,data,label,attr): 
        leaf.feature = attr
        leaf.values = set(data.iloc[leaf.index][leaf.feature])
        indexes = data.iloc[leaf.index].groupby([leaf.feature]).groups
        for name, index in indexes.items():
            classes = dict(data.iloc[index][label].value_counts())
            if len(classes) > 0:
                node = Node(index,classes,label,depth=leaf.depth+1)
                leaf.subnode(node,name)
                self.leaves.append(node)
        self.leaves.remove(leaf)
        
    
    def predict(self,data):
        predict = []
        for i in data.index:
  
            start = self.head
            while start.feature!=None:
                val = data.iloc[i][start.feature]
                if val in start.valueNode.keys():
                    start = start.valueNode[val]
                else:
                    break
            
            bClass = int()
            nClass = -1
            
            for i,j in start.classes.items():
                if j>nClass:
                    nClass = j
                    bClass = i
            predict.append(bClass)
        return np.array(predict)
    
    def calc_accuracy(self,predict,label):
        return (sum(label==predict)/len(label))
        

In [7]:
# calc_info_gain(train_df,'Survived','Age')

# train_df[['Survived']].apply(calc_entropy)

# for name,index in train_df.groupby(['Age']).groups.items():
#     print(name,train_df.iloc[index][['Survived']].apply(calc_entropy))
    

# sum(train_df.groupby(['Age'])['Survived'].count()/sum(train_df.groupby(['Age'])['Survived'].count())\
# *train_df.groupby(['Age'])['Survived'].apply(calc_entropy))

In [8]:
def calc_info_gain(data,label,feature):
    entropyS = data[[label]].apply(calc_entropy)
    featureCount = data.groupby([feature]).count().values[:,0]
    weights = featureCount/sum(featureCount)
    en = sum(data.groupby([feature])[label].apply(calc_entropy)* weights)
    gain = entropyS - en
    return gain[0]
    

In [33]:
def calc_Gini(data):
    values = set(data)
    count = {}
    for value in values:
        count[value] = 0
    for i in data:
        count[i]+=1
    counts = np.array(list(count.values()))
    p =list(counts/sum(counts))
    gini = 1
    for i in p:
        if i!=0:
            gini += -i**2   
    return gini

In [10]:
def calc_entropy(data):
    values = set(data)
    count = {}
    for value in values:
        count[value] = 0
    for i in data:
        count[i]+=1
    counts = np.array(list(count.values()))
    p =list(counts/sum(counts))
    e = 0
    for i in p:
        if i!=0:
            e += -i * np.log2(i)     
    return e

In [11]:
def best_feature(dic):
    max_ = np.max(list(dic.values()))
    for key,value in dic.items():
        if value == max_:
            return key
    return None

In [12]:
def build_tree(data,features,label,max_depth):
    classes = dict(data[label].value_counts())
    tree = Tree(data,classes,label,max_depth)
    position = 0
    
    while position <len(tree.leaves):

        leaf = tree.leaves[position]
        if (len(leaf.classes)<=1) or (len(data.iloc[leaf.index])<=1) or (leaf.depth >=tree.max_depth):
            position += 1
        else:
            dic_gain = {}
            for feature in features:
                info_gain = calc_info_gain(data[[feature,label]],label,feature)
                if (info_gain >5e-4) and (data.iloc[leaf.index][[feature]].apply(calc_entropy)[0]!=0):
                    dic_gain[feature] = info_gain
                    
            if len(dic_gain) >0:
                bFeature = best_feature(dic_gain)
                tree.branch(leaf,data,label,bFeature)
            else:
                position += 1
    return tree

In [13]:
# # Test tree
# a=Tree(df,['bad','good','normal'],'label')
# label = 'label'
# features = set(df.columns)-{'label'}
# dic_gain = {}
# for feature in features:
#     dic_gain[feature] = calc_info_gain(df[[feature,label]],label,feature)
# name = -1
# m = 0
# for i ,j in dic_gain.items():
#     if j>m:
#         m = j
#         name = i
# best_feature = name
# a.branch(tree.head,df,label,best_feature)

In [14]:
def gender_status_(name):
    if 'Mr.' in name:
        return 'Mr'
    elif 'Mrs.' in name:
        return 'Mrs'
    elif 'Miss.' in name:
        return 'Miss'
    else:
        return None

def title_(name):
    if 'Master.' in name:
        return 'Master'
    elif 'Dr.' in name:
        return 'Dr'
    else:
        return 'normal'

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv('data/test.csv')

In [15]:

train_df['gender_status'] = train_df['Name'].apply(gender_status_)
test_df['gender_status'] = test_df['Name'].apply(gender_status_)

train_df['title'] = train_df['Name'].apply(title_)
test_df['title'] = test_df['Name'].apply(title_)



train_df[['Embarked']] = train_df[['Embarked']].fillna(train_df['Embarked'].mode()[0])
train_df[['Age']] = train_df[['Age']].fillna(train_df['Age'].mean())
train_df[['gender_status']] = train_df[['gender_status']].fillna(train_df['gender_status'].mode())
train_df[['title']] = train_df[['title']].fillna(train_df['title'].mode())


test_df[['Embarked']] = test_df[['Embarked']].fillna(train_df['Embarked'].mode()[0])

test_df[['Age']] = test_df[['Age']].fillna(train_df['Age'].mean())
test_df[['gender_status']] = test_df[['gender_status']].fillna(train_df['gender_status'].mode())
test_df[['title']] = test_df[['title']].fillna(train_df['title'].mode())
train_df = train_df.drop(['Cabin','PassengerId','Name','Fare','Ticket','Sex','title'],axis=1)
test_df = test_df.drop(['Cabin','PassengerId','Name','Fare','Ticket','Sex','title'],axis=1)







train_df[['Parch']]= pd.cut(train_df['Parch'],3,labels=[1,2,3])
train_df[['Age']]= pd.cut(train_df['Age'],4,labels=[1,2,3,4])
train_df[['SibSp']] = pd.cut(train_df['SibSp'],3,labels=[1,2,3])


test_df[['Parch']]= pd.cut(test_df['Parch'],3,labels=[1,2,3])
test_df[['Age']]= pd.cut(test_df['Age'],4,labels=[1,2,3,4])
test_df[['SibSp']] = pd.cut(test_df['SibSp'],3,labels=[1,2,3])

In [16]:
# def categorical_age(x):
#     if x<16:
#         return 'Child'
#     elif x<40:
#         return 'young'
#     elif x<60:
#         return 'growup'
#     else:
#         return 'old'

# train_df[['Age']] = train_df[['Age']].applymap(categorical_age)
# test_df[['Age']] = test_df[['Age']].fillna(test_df.mean())
# test_df[['Age']] = test_df[['Age']].applymap(categorical)

In [17]:
tree = build_tree(train_df,set(train_df.columns)-{'Survived'},'Survived',6)
tree.calc_accuracy(tree.predict(train_df),train_df['Survived'])

0.7631874298540965

In [18]:
sub = pd.read_csv("data/gender_submission.csv")
sub = sub.drop(['PassengerId'],axis=1)

In [19]:
data = {k:[v] for k,v in dict(test_df.iloc[48]).items()}
dataframe = pd.DataFrame(data = data)
# for i in dataframe.index:
#     print(i)
#     print(dataframe.iloc[i])
tree.predict(dataframe)

array([1])

In [20]:
tree.calc_accuracy(tree.predict(test_df),sub['Survived'])

0.6578947368421053

In [21]:
from graphviz import Digraph

In [22]:
dot = Digraph(comment='The Round Table')

In [23]:
def draw_graph(x,dot):
    parent = []
    dot.node('%s'%(x.index),'%s\n %s'%(x.feature,x.classes))
    parent.append(x)
    dic ={0:'Dead',1:'Alive'}
    dot.attr('node', shape='box')
    while len(parent)>0:
        
        element = parent[0]
        parent = parent[1:]
        if len(element.valueNode)>0:
            for i,sub in element.valueNode.items():
                if (sub.feature!=None):
                    dot.node('%s'%(sub.index),'%s\n %s'%(sub.feature,sub.classes))
                else:
                    
                    dot.node('%s'%(sub.index),'%s\n %s'%(dic[best_feature(sub.classes)],sub.classes))
                dot.edge('%s'%(element.index),'%s'%(sub.index),label=str(i))
                parent.append(sub)
        else:
            continue
            
    return dot
                   

In [24]:
dot = Digraph(comment='Decision Tree')
dot = draw_graph(tree.head,dot)

In [25]:
dot.render('test-output/round-table.gv', view=True)

'test-output/round-table.gv.pdf'

In [26]:
for i in tree.leaves:
    if (i.classes)=={1:9,0:9}:
        print(train_df.iloc[i.index])

In [27]:
def cross_validation(df,k_fold,max_depth):
    size = int(len(df)/k_fold)
    result = []
    for i in range(k_fold):
        if i < k_fold-1:
            index_train = [x for x in range(len(df)) if x< i*size or x >=(i+1)*size]
            index_test = [x for x in range(i*size,(i+1)*size+1)]
        else:
            index_train = [x for x in range(len(df)) if x<i*size ]
            index_test = [x for x in range(i*size,len(df))]
            
            tree = build_tree(df.iloc[index_train],set(df.columns)-{'Survived'},'Survived',max_depth)
            result.append(tree.calc_accuracy(tree.predict(df.iloc[index_test].reset_index(drop=True)),df.iloc[index_test]['Survived']))
    return np.mean(result)
        
    
    
    

In [28]:
for max_depth in range(1,10):
    print(cross_validation(train_df,5,max_depth))

0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978
0.6927374301675978


In [29]:
tree.predict(train_df.iloc[[1,2,3,4,5]].reset_index(drop=True))

array([1, 0, 1, 0, 0])

In [30]:
train_df.iloc[[1,2,3,4,5]].reset_index(drop=True)

,Survived,Pclass,Age,SibSp,Parch,Cabin,Embarked,gender_status
0,1,1,2,1,1,C85,C,Mrs
1,1,3,2,1,1,NaN,S,Miss
2,1,1,2,1,1,C123,S,Mrs
3,0,3,2,1,1,NaN,S,Mr
4,0,3,2,1,1,NaN,Q,Mr


In [31]:
tree = build_tree(train_df,set(train_df.columns)-{'Survived'},'Survived',5)
tree.calc_accuracy(tree.predict(test_df),sub['Survived'])

0.6578947368421053

In [32]:
dot = Digraph(comment='Decision Tree')
dot = draw_graph(tree.head,dot)
dot.render('test-output/round-table.gv', view=True)

'test-output/round-table.gv.pdf'